# Mini-project II: Text Classification
### build model from scratch

### Load file

In [15]:
class File:
    def read_raw_text_file(file_path):
        with open(file_path, 'r', encoding='latin-1') as file: 
            FileContent = file.read()
            file.close()
        return FileContent
    
    def write_text_file(file_path, text: str):
        print("write text with string")
        with open(file_path, 'w') as writefile:
            writefile.write(text)
            writefile.close()
    
    def write_text_file(file_path, texts: list):
        txt: str = ''
        for tx in texts:
            txt += (tx + '\n')

        with open(file_path, 'w') as writefile:
            writefile.write(txt)
            writefile.close()

In [16]:

negative_words = File.read_raw_text_file('corpus/negative-words.txt')
negative_words = negative_words.split()
print(len(negative_words))
positive_words = File.read_raw_text_file('corpus/positive-words.txt')
positive_words = positive_words.split()

4780


In [17]:
negative_reviews = File.read_raw_text_file('corpus/negative-reviews.txt')
positive_reviews = File.read_raw_text_file('corpus/positive-reviews.txt')


In [18]:
print(len(positive_reviews))
positive_reviews = positive_reviews.splitlines()
negative_reviews = negative_reviews.splitlines()

946633


In [11]:
# log
positive_reviews

'Size, Size, ans Size.\nExcellent quality, speedy printing, low cost\nCheap, good quality, small size\nAttractive design, satisfying features, the backlight!\nPretty much has every feature you could possibly need, great look\nEase of use, small size and reliability\ncheap,good print quality\ntakes clear shots\nSmall, Light, Stylish, backlight\nAbsolutely Outstanding Photo Printing Results.  Easy to Use.\nGreat signal, durable, compact\nCompact, easy to navigate menu,\nLight, Feature Packed, Customizable, Easily fits in pocket\nsmall and lightweight, easy to use\nEasy to use feature loaded.\nExcellent quality, user friendly, compact\nHonestly the best phone I have owned. Voice dialing!\nTakes great photos\nvery clear photos\nNone to speak of.\nColorful, voice recognition\nNone!\nStyle, Features.\nwhitish blue backlight color, animated menus, clear sound, cheep price, pretty light, etc.\nMany different functions from one machine\nSmall\nEasy, very portable, has everything I need\nJust ab

In [19]:
total_positive_review = len(positive_reviews)
total_negative_review = len(negative_reviews)

train_size_pos = int(0.8 * total_positive_review)
test_size_pos = int(0.2 * total_positive_review)

train_size_neg = int(0.8 * total_negative_review)
test_size_neg = int(0.2 * total_negative_review)

In [22]:
print("Training pos and negative size total: ", train_size_pos, train_size_neg)
train_positive_set = positive_reviews[:train_size_pos]
test_positive_set = positive_reviews[train_size_pos:]
train_negative_set = negative_reviews[:train_size_neg]
test_negative_set = negative_reviews[train_size_neg:]

Training pos and negative size total:  16000 16000


In [26]:
# train_negative_set
len(train_negative_set)

16000

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter

[nltk_data] Downloading package punkt to /Users/rattanak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# positive_train_count = [char for char in tokens]
# uni_counts = Counter(unigram)


# tokenized_train_data = [word_tokenize(sentence) for sentence in train_positive_set]
# tokenized_train_data = [word for word in tokenized_train_data]
# print(tokenized_train_data)

# Tokenize each sentence and flatten the list of words
tokenized_train_pos_data = [word.lower().strip('.,!') for sentence in train_positive_set for word in sentence.split()]
tokenized_train_neg_data = [word.lower().strip('.,!') for sentence in train_negative_set for word in sentence.split()]

# Count word frequencies
word_counts_pos_train = Counter(tokenized_train_pos_data)
word_counts_neg_train = Counter(tokenized_train_neg_data)
print(word_counts_neg_train)
# unigram = [char for char in tokens]
# uni_counts = Counter(unigram)

Counter({'to': 2459, 'no': 2168, 'a': 2131, 'not': 1910, 'the': 1901, 'battery': 1850, 'is': 1477, 'and': 1442, 'of': 1295, 'life': 1170, 'for': 1024, 'none': 977, 'slow': 959, 'poor': 928, 'quality': 900, 'in': 894, 'expensive': 813, 'ink': 727, 'be': 724, 'with': 722, 'on': 707, 'it': 700, 'small': 629, 'too': 626, 'little': 610, 'very': 603, 'batteries': 537, 'phone': 530, 'zoom': 514, 'are': 514, 'only': 502, 'as': 476, 'flash': 473, 'paper': 466, 'camera': 465, 'memory': 463, 'you': 455, 'price': 452, 'low': 443, 'software': 435, 'hard': 434, 'can': 426, 'i': 422, 'use': 419, 'bad': 418, 'screen': 410, 'up': 384, 'some': 383, 'have': 382, 'time': 379, 'bit': 364, 'cartridges': 358, 'when': 347, 'short': 347, 'but': 338, "doesn't": 330, 'out': 325, 'could': 322, 'reception': 318, 'at': 317, 'than': 317, 'bulky': 315, 'that': 311, 'features': 307, 'pictures': 303, 'card': 295, 'lens': 294, 'color': 292, 'size': 284, 'or': 281, 'print': 277, 'picture': 273, 'lcd': 272, 'printing': 26

### Multinomial Naive Bayes Classifier

In [9]:
# Prior probability
prob_positive_class = len(train_positive_set) /  len(train_negative_set + train_positive_set)
prob_negative_class = len(train_negative_set) / len(train_negative_set + train_positive_set)

In [10]:
from typing import Final

# Likelihood
VOCAB_SIZE: Final[float] = len(word_counts_neg_train) + len(word_counts_pos_train)
NEGATIVE: Final[int] = 0
POSITIVE: Final[int] = 1

def prob_positive(word: str):
    """ 
        add k-smoothing to avoid zero probability
        Returns:
        float: Smoothed probability of the word in the positive class.
    """
    word_count = word_counts_pos_train.get(word)
    if not word_count:
        word_count = 0
    # print("count: ", word_count)
    return ((word_count + 1) * 1.0) / (len(train_positive_set) +  VOCAB_SIZE)

def prob_negative(word: str):
    # add k-smoothing to avoid zero probability
    word_count = word_counts_neg_train.get(word)
    if not word_count:
        word_count = 0
    # print("count: ", word_count)
    return ((word_count + 1) * 1.0) / (len(train_negative_set) +  VOCAB_SIZE)

In [11]:
# len(word_counts_neg_train)
print(test_positive_set[:1])

def predict_positive(sentence: str):
    # prob = []
    total_prob = 0
    for word in sentence.split():
        # prob_positive(word)
        total_prob += prob_positive(word)
        # prob.append(prob_positive(word))
    return prob_positive_class * total_prob

def predict_negative(sentence: str):
    # prob = []
    total_prob = 0
    for word in sentence.split():
        # prob_positive(word)
        total_prob += prob_negative(word)
        # prob.append(prob_positive(word))
    return prob_negative_class * total_prob

['Strong flash, direct TV connection, Free photo editing software, above average pic quality']


In [12]:
def multinomialNBTextClassification(sentence: str):
    result: int = NEGATIVE
    prob_pos = predict_positive(sentence)
    prob_neg = predict_negative(sentence)
    if prob_pos > prob_neg:
        result = POSITIVE

    return result

In [15]:
import pandas as pd
multinomialNBTextClassification(test_positive_set[:1][0])

def predict(test_sets):
    df = pd.DataFrame(columns=['text', 'class'])
    true_count = 0
    false_count = 0
    for sentence in test_sets:
        res = multinomialNBTextClassification(sentence)
        if res == POSITIVE:
            true_count += 1
        else:
            false_count +=1
        df.loc[len(df)] =  [sentence, res]

    return {
        "true_count": true_count,
        "false_count": false_count,
        "data": df
    }

In [16]:
positive_test = predict(test_positive_set)
negative_test = predict(test_negative_set)
accuracy = (positive_test['true_count'] + negative_test['true_count']) / (positive_test['true_count'] + negative_test['true_count'] + positive_test['false_count'] + negative_test['false_count'])

In [21]:
print("accuracy: ", accuracy)
print("TP: ", positive_test['true_count'], " TN: ", positive_test['false_count'])
print("FP: ", negative_test['true_count'], "FN: ", negative_test['false_count'])

accuracy:  0.52475
TP:  3017  TN:  983
FP:  1181 FN:  2819
